In [ ]:
#Trafficarticle4
import os
import shutil
import numpy as np
import librosa
import soundfile as sf
from scipy.signal import stft

# Paths and Configuration
OUTPUT_DIR = "processed_data_traffic_article4"  # Updated output directory for traffic data
CLEAN_AUDIO_PATH = r"C:\Users\enhance\converted_clean_wav"  # Clean audio folder
NOISE_PARENT_DIR = r"C:\Users\enhance\Downloads\traffic_noise_wav"  # Updated folder for traffic noise
SAMPLE_RATE = 16000  # Sampling rate
FRAME_LENGTH = 512  # Frame length in samples (32 ms)
FRAME_SHIFT = 128  # Frame shift in samples
SNRS = [0, 5, 10, 15]  # Signal-to-noise ratios for augmentation
TARGET_FREQUENCY_BINS = 257  # Target frequency bins (based on article)
TARGET_TIME_FRAMES = 128  # Target number of time frames (adjust as needed)

# Function to clear the output folder
def clear_folder(folder_path):
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)
        print(f"Cleared folder: {folder_path}")
    os.makedirs(folder_path, exist_ok=True)
    print(f"Recreated folder: {folder_path}")

# Function to recursively extract audio files from a directory
def extract_files(parent_dir, extension=".wav"):
    files = []
    for root, _, filenames in os.walk(parent_dir):
        for file in filenames:
            if file.endswith(extension):
                files.append(os.path.join(root, file))
    print(f"Found {len(files)} {extension} files in {parent_dir}.")
    return files

# Function to add noise at a specific SNR
def add_noise(clean_audio, noise_audio, snr):
    if len(noise_audio) > len(clean_audio):
        start_idx = np.random.randint(0, len(noise_audio) - len(clean_audio))
        noise_audio = noise_audio[start_idx:start_idx + len(clean_audio)]
    else:
        noise_audio = np.pad(noise_audio, (0, len(clean_audio) - len(noise_audio)), mode='wrap')
    clean_power = np.mean(clean_audio ** 2)
    noise_power = np.mean(noise_audio ** 2)
    scaling_factor = np.sqrt(clean_power / (10 ** (snr / 10) * noise_power))
    noisy_audio = clean_audio + scaling_factor * noise_audio
    return noisy_audio

# Function to compute and pad log-power spectrogram
def compute_log_power_spectrogram(audio):
    _, _, Zxx = stft(audio, fs=SAMPLE_RATE, nperseg=FRAME_LENGTH, noverlap=FRAME_LENGTH - FRAME_SHIFT, window='hann')
    lps = np.log(np.abs(Zxx) ** 2 + 1e-10)

    # Pad or truncate frequency bins
    if lps.shape[0] < TARGET_FREQUENCY_BINS:
        padding_freq = TARGET_FREQUENCY_BINS - lps.shape[0]
        lps = np.pad(lps, ((0, padding_freq), (0, 0)), mode='constant')
    elif lps.shape[0] > TARGET_FREQUENCY_BINS:
        lps = lps[:TARGET_FREQUENCY_BINS, :]

    # Pad or truncate time frames
    if lps.shape[1] < TARGET_TIME_FRAMES:
        padding_time = TARGET_TIME_FRAMES - lps.shape[1]
        lps = np.pad(lps, ((0, 0), (0, padding_time)), mode='constant')
    elif lps.shape[1] > TARGET_TIME_FRAMES:
        lps = lps[:, :TARGET_TIME_FRAMES]

    return lps

# Function to preprocess audio
def preprocess_audio(clean_path, noise_files, output_dir, snrs):
    clean_audio, _ = librosa.load(clean_path, sr=SAMPLE_RATE)
    for snr in snrs:
        noise_path = np.random.choice(noise_files)
        noise_audio, _ = librosa.load(noise_path, sr=SAMPLE_RATE)
        noisy_audio = add_noise(clean_audio, noise_audio, snr)
        clean_lps = compute_log_power_spectrogram(clean_audio)
        noisy_lps = compute_log_power_spectrogram(noisy_audio)

        # Save noisy audio and spectrograms
        snr_folder = os.path.join(output_dir, f"SNR_{snr}")
        os.makedirs(snr_folder, exist_ok=True)

        base_name = os.path.splitext(os.path.basename(clean_path))[0]
        sf.write(os.path.join(snr_folder, f"{base_name}_noisy_audio.wav"), noisy_audio, SAMPLE_RATE)
        np.save(os.path.join(snr_folder, f"{base_name}_clean_lps.npy"), clean_lps)
        np.save(os.path.join(snr_folder, f"{base_name}_noisy_lps.npy"), noisy_lps)

# Main function
def main():
    clear_folder(OUTPUT_DIR)
    clean_files = extract_files(CLEAN_AUDIO_PATH, extension=".wav")
    noise_files = extract_files(NOISE_PARENT_DIR, extension=".wav")

    if not clean_files:
        print(f"No clean .wav files found in {CLEAN_AUDIO_PATH}. Please check the directory.")
        return

    if not noise_files:
        print(f"No noise .wav files found in {NOISE_PARENT_DIR}. Please check the directory.")
        return

    for clean_file in clean_files:
        preprocess_audio(clean_file, noise_files, OUTPUT_DIR, SNRS)
        print(f"Processed: {os.path.basename(clean_file)}")

if __name__ == "__main__":
    main()


Recreated folder: processed_data_traffic_article4
Found 2703 .wav files in C:\Users\enhance\converted_clean_wav.
Found 47 .wav files in C:\Users\enhance\Downloads\traffic_noise_wav.
Processed: 1272-128104-0000.wav
Processed: 1272-128104-0001.wav
Processed: 1272-128104-0002.wav
Processed: 1272-128104-0003.wav
Processed: 1272-128104-0004.wav
Processed: 1272-128104-0005.wav
Processed: 1272-128104-0006.wav
Processed: 1272-128104-0007.wav
Processed: 1272-128104-0008.wav
Processed: 1272-128104-0009.wav
Processed: 1272-128104-0010.wav
Processed: 1272-128104-0011.wav
Processed: 1272-128104-0012.wav
Processed: 1272-128104-0013.wav
Processed: 1272-128104-0014.wav
Processed: 1272-135031-0000.wav
Processed: 1272-135031-0001.wav
Processed: 1272-135031-0002.wav
Processed: 1272-135031-0003.wav
Processed: 1272-135031-0004.wav
Processed: 1272-135031-0005.wav
Processed: 1272-135031-0006.wav
Processed: 1272-135031-0007.wav
Processed: 1272-135031-0008.wav
Processed: 1272-135031-0009.wav
Processed: 1272-13

In [5]:
import os
import numpy as np

def get_input_shapes(processed_data_dir):
    """
    Get the shapes of all .npy files in the processed data directory.
    """
    shapes = []  # List to store shapes of all files
    for root, _, files in os.walk(processed_data_dir):
        for file in files:
            if file.endswith(".npy"):  # Look for .npy files
                file_path = os.path.join(root, file)
                try:
                    data = np.load(file_path)  # Load the .npy file
                    shapes.append((file, data.shape))  # Record the shape of the array
                except Exception as e:
                    print(f"Error loading file {file_path}: {e}")

    return shapes

# Directory containing preprocessed files
processed_data_dir = "processed_data_traffic_article4" 

# Get the shapes of all processed .npy files
input_shapes = get_input_shapes(processed_data_dir)

# Print the shapes of all processed .npy files
if input_shapes:
    print("Shapes of processed .npy files:")
    for file_name, shape in input_shapes:
        print(f"{file_name}: {shape}")
else:
    print("No .npy files found in the processed data directory!")
unique_shapes = set(input_shapes)
print(f"Unique shapes found: {unique_shapes}")



Shapes of processed .npy files:
1272-128104-0000_clean_lps.npy: (257, 128)
1272-128104-0000_noisy_lps.npy: (257, 128)
1272-128104-0001_clean_lps.npy: (257, 128)
1272-128104-0001_noisy_lps.npy: (257, 128)
1272-128104-0002_clean_lps.npy: (257, 128)
1272-128104-0002_noisy_lps.npy: (257, 128)
1272-128104-0003_clean_lps.npy: (257, 128)
1272-128104-0003_noisy_lps.npy: (257, 128)
1272-128104-0004_clean_lps.npy: (257, 128)
1272-128104-0004_noisy_lps.npy: (257, 128)
1272-128104-0005_clean_lps.npy: (257, 128)
1272-128104-0005_noisy_lps.npy: (257, 128)
1272-128104-0006_clean_lps.npy: (257, 128)
1272-128104-0006_noisy_lps.npy: (257, 128)
1272-128104-0007_clean_lps.npy: (257, 128)
1272-128104-0007_noisy_lps.npy: (257, 128)
1272-128104-0008_clean_lps.npy: (257, 128)
1272-128104-0008_noisy_lps.npy: (257, 128)
1272-128104-0009_clean_lps.npy: (257, 128)
1272-128104-0009_noisy_lps.npy: (257, 128)
1272-128104-0010_clean_lps.npy: (257, 128)
1272-128104-0010_noisy_lps.npy: (257, 128)
1272-128104-0011_clean